In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/기계학습/skku-2023-1-machine-learning-first-project/train/bulk_115s_01.csv")

In [ ]:
import os
files = os.listdir('/content/drive/MyDrive/기계학습/skku-2023-1-machine-learning-first-project/train')

In [ ]:
files

['bulk_115s_01.csv',
 'bulk_130s_01.csv',
 'bulk_170s_01.csv',
 'bulk_xs_01.csv',
 'bulk_xs_06.csv',
 'bulk_xs_07.csv',
 'bulk_xs_02.csv',
 'bulk_xs_05.csv',
 'bulk_xs_04.csv',
 'bulk_xs_03.csv',
 'bulk_xs_09.csv',
 'bulk_xs_12.csv',
 'bulk_xs_10.csv',
 'bulk_xs_11.csv',
 'bulk_xs_08.csv',
 'bulk_xs_16.csv',
 'interactive_07.csv',
 'interactive_shell_05.csv',
 'interactive_shell_17.csv',
 'interactive_shell_18.csv',
 'interactive_14.csv',
 'interactive_shell_11.csv',
 'web_1page_07.csv',
 'web_3page_04.csv',
 'video_210s480p_01.csv',
 'video_180s720p_07.csv',
 'web_3page_02.csv',
 'interactive_18.csv',
 'interactive_shell_09.csv',
 'interactive_03.csv',
 'web_multiple_05.csv',
 'interactive_shell_01.csv',
 'web_1page_04.csv',
 'video_180s720p_04.csv',
 'web_3page_01.csv',
 'web_2page_06.csv',
 'interactive_04.csv',
 'interactive_17.csv',
 'video_180s480p_02.csv',
 'interactive_shell_14.csv',
 'interactive_shell_02.csv',
 'video_180s1080p_04.csv',
 'interactive_02.csv',
 'web_2page_01.c

In [ ]:
#파일명 마다 데이터 프레임 할당
file_df=[]

for value in files:
  data = pd.read_csv(f"/content/drive/MyDrive/기계학습/skku-2023-1-machine-learning-first-project/train/{value}")
  file_df.append(data)

In [ ]:
for df in file_df:
  df['time'] = pd.to_datetime(df['time'],unit='s')
  df['time'] = df['time'].dt.strftime('%Y/%m/%d %H:%M:%S')

In [ ]:
file_df[0].head()

,time,proto,data_len,ip_src,ip_dst,src_port,dst_port
0,2019/03/01 11:10:58,17,58,192.168.1.149,192.168.1.1,52835,53
1,2019/03/01 11:10:59,17,230,192.168.1.1,192.168.1.149,53,52835
2,2019/03/01 11:10:59,6,603,192.168.1.149,80.249.99.148,51850,80
3,2019/03/01 11:10:59,6,1448,80.249.99.148,192.168.1.149,80,51850
4,2019/03/01 11:10:59,6,1448,80.249.99.148,192.168.1.149,80,51850


In [ ]:
#file_df에 있는 데이터 프레임 groupby 진행
df_gb = [] #브로드캐스팅 완료한 df 리스트

for df in file_df:
  df = df.groupby(['time','ip_dst'],)['data_len'].count().reset_index() #time과 ip_dst로 groupby한 뒤 갯수를 세주기 위해 count
  df = df.groupby(['ip_dst']).filter(lambda x:len(x)>40).reset_index() #ip_dst로 groupby, filter하여 그 값의 count가 40이이 넘는 값만 남
  df_gb.append(df)

In [ ]:
t_columns = ['traffic(t-10)','traffic(t-9)','traffic(t-8)','traffic(t-7)','traffic(t-6)','traffic(t-5)','traffic(t-4)','traffic(t-3)','traffic(t-2)','traffic(t-1)','traffic(t)']

interactive = pd.DataFrame(columns = t_columns)
bulk = pd.DataFrame(columns = t_columns)
video = pd.DataFrame(columns = t_columns)
web = pd.DataFrame(columns = t_columns)

sample = pd.DataFrame(columns = t_columns)

for df,file in zip(df_gb,files):
  df = pd.DataFrame(df.groupby('ip_dst')['data_len']). #ip_dst별로 다시 groupby하여 각 ip_dst의 data_len 값을 불러
  if 'bulk' in file: #파일명에 따라 각 라벨 별로 분류
    for i in range(df.shape[0]): #그 파일 안의 ip_dst의 개수
      data_len = df.iloc[i,1].to_list() #리스트로 변경
      for j in range(len(data_len)-10): #data_len - window_size + 1 만큼 반복
        temp = pd.DataFrame(data=[data_len[j:j+11]],columns = t_columns) #11개씩 값을 가져옴  #t_columns를 column 값으로 하는 데이터 프레임으로 생성
        bulk = pd.concat([bulk,temp],ignore_index=True) #각 label별 데이터 프레임에 concat
  elif 'interactive' in file:
    for i in range(df.shape[0]):
      data_len = df.iloc[i,1].to_list()
      for j in range(len(data_len)-10):
        temp = pd.DataFrame(data=[data_len[j:j+11]],columns = t_columns) 
        interactive = pd.concat([interactive,temp],ignore_index=True)
  elif 'video' in file:
    for i in range(df.shape[0]):
      data_len = df.iloc[i,1].to_list()
      for j in range(len(data_len)-10):
        temp = pd.DataFrame(data=[data_len[j:j+11]],columns = t_columns) 
        video = pd.concat([video,temp],ignore_index=True)
  else:
    for i in range(df.shape[0]):
      data_len = df.iloc[i,1].to_list()
      for j in range(len(data_len)-10):
        temp = pd.DataFrame(data=[data_len[j:j+11]],columns = t_columns) 
        web = pd.concat([web,temp],ignore_index=True)


In [ ]:
video

,traffic(t-10),traffic(t-9),traffic(t-8),traffic(t-7),traffic(t-6),traffic(t-5),traffic(t-4),traffic(t-3),traffic(t-2),traffic(t-1),traffic(t)
0,251,2161,48,111,190,3,3,1218,217,1346,3
1,2161,48,111,190,3,3,1218,217,1346,3,304
2,48,111,190,3,3,1218,217,1346,3,304,1285
3,111,190,3,3,1218,217,1346,3,304,1285,3
4,190,3,3,1218,217,1346,3,304,1285,3,293
...,...,...,...,...,...,...,...,...,...,...,...
903,1521,1643,514,674,1326,1,1198,3,1301,1285,456
904,1643,514,674,1326,1,1198,3,1301,1285,456,996
905,514,674,1326,1,1198,3,1301,1285,456,996,559
906,674,1326,1,1198,3,1301,1285,456,996,559,962


In [ ]:
interactive['label'] = 0
bulk['label'] = 1
video['label']= 2
web['label'] = 3

train = pd.concat([interactive,bulk,video,web],ignore_index=True)

In [ ]:
train.to_csv('train_none_index.csv',mode='w',index=None)

In [ ]:
train

,traffic(t-10),traffic(t-9),traffic(t-8),traffic(t-7),traffic(t-6),traffic(t-5),traffic(t-4),traffic(t-3),traffic(t-2),traffic(t-1),traffic(t),label
0,1,18,3,4,10,3,7,4,20,7,8,0
1,18,3,4,10,3,7,4,20,7,8,14,0
2,3,4,10,3,7,4,20,7,8,14,14,0
3,4,10,3,7,4,20,7,8,14,14,14,0
4,10,3,7,4,20,7,8,14,14,14,7,0
...,...,...,...,...,...,...,...,...,...,...,...,...
12640,3,4,19,1,1,27,9,1,3,5,24,3
12641,4,19,1,1,27,9,1,3,5,24,2,3
12642,19,1,1,27,9,1,3,5,24,2,3,3
12643,1,1,27,9,1,3,5,24,2,3,95,3
